In [1]:
try:
    with open("global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [2]:
import random
import tensorflow as tf
import numpy as np
import re
import pprint
from src.wikipedia import Wikipedia
#random.seed(12345)

In [3]:
def distortText(text, swap_type = "symboldelete"):
       
    # Distortion types
    distortions_symbols = ["symbolswap","symboldelete","symbolreplacerandom","symbolreplaceprev"]
    distortions_words = ["wordtrimright", "wordswapprev"]
    distortions_sentences = ["deletespaces"]
    # Symbol operations: max 2 symbols per word, max 2 symbol distortion types per word
    # Word trim: max 1 per word
    # Delete spaces: max all spaces, min 0 spaces
    
    list_words = text.split()
    
    distortion_prob_symbol = 0.5
    for i in range(len(list_words)):
        r = random.random()
        if swap_type == "symbolswap":
            # SYMBOL SWAP
            ## min distorted symbol = 0, max distorted symbol = len-2 because of indexing with 0 and this is the 1st swappable symbol
            distorted_symbol = round((len(list_words[i]) - 2) * random.random())
            s1 = list_words[i][distorted_symbol + 1]
            #print("{}---{}-{}-{}-{}".format(distorted_symbol, list_words[i][0:distorted_symbol - 1], list_words[i][distorted_symbol], s1, list_words[i][distorted_symbol + 1:]))
            #print(list_words[i][0:(distorted_symbol - 1) * (distorted_symbol > 0)])
            list_words[i] = list_words[i][0:distorted_symbol] + s1 + list_words[i][distorted_symbol] + list_words[i][distorted_symbol + 2:]
        elif swap_type == "symboldelete":
            # SYMBOL DELETE
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][0:distorted_symbol] + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplacerandom":
            symbols = "abcdefghijklmnopqrstuvwxyz"
            # SYMBOL REPLACE RANDOM
            ## min distorted symbol = 0, max distorted symbol = len-1
            distorted_symbol = round((len(list_words[i]) - 1) * random.random())
            replace = symbols[round((len(symbols) - 1) * random.random())]
            list_words[i] = list_words[i][0:distorted_symbol] + replace + list_words[i][distorted_symbol + 1:]
        elif swap_type == "symbolreplaceprev":
            # SYMBOL REPLACE PREVIOUS
            ## min distorted symbol = 1, max distorted symbol = len - 1
            distorted_symbol = round(1 + (len(list_words[i]) - 2) * random.random())
            #print("{} - {}".format(distorted_symbol, list_words[i]))
            #print("{}---{}-{}-{}".format(distorted_symbol, list_words[i][0:(distorted_symbol)], list_words[i][distorted_symbol - 1], list_words[i][distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol):distorted_symbol + 1 * (len(list_words[i]) > distorted_symbol)]))
            list_words[i] = list_words[i][0:(distorted_symbol)] + list_words[i][distorted_symbol - 1] + list_words[i][distorted_symbol + 1:]
        
        if swap_type == "wordtrimright":
            # WORD TRIM RIGHT
            ## min symbol = 1, max distorted symbol = len - 1
            trim_from = round(1 + (len(list_words[i]) - 1) * random.random())
            list_words[i] = list_words[i][:trim_from]
    
    if swap_type == "wordswapprev":
        # WORD SWAP
        word_id = round((len(list_words) - 2) * random.random())
        s1 = list_words[word_id]
        list_words[word_id] = list_words[word_id + 1]
        list_words[word_id + 1] = s1
        
    if swap_type == "deletespaces":
        # DELETE WHITESPACES
        ## -2 because there are len - 1 spaces in total
        space_id = round((len(list_words) - 2) * random.random())
        list_words[space_id] = list_words[space_id] + list_words[space_id + 1]
        list_words.pop(space_id + 1)
            
    distortedText = " ".join(list_words)
    return distortedText

In [ ]:
text = "Antons likes pizza"
print(distortText(text=text,swap_type="symbolswap"))

print(distortText(text=text,swap_type="symboldelete"))

print(distortText(text=text,swap_type="symbolreplacerandom"))

print(distortText(text=text,swap_type="symbolreplaceprev"))

print(distortText(text=text,swap_type="wordtrimright"))

print(distortText(text=text,swap_type="wordswapprev"))

print(distortText(text=text,swap_type="deletespaces"))

In [4]:
wikipedia = Wikipedia(
    language="simple",
    cache_directory_url=False
)

simplewiki-latest-pages-articles-multistream.xml.bz2
Loading parsed documents.
Loading preprocessed documents.
Wikipedia loaded.


In [134]:
# Cleaning up simple wikipedia texts
pattern_ignored_words = re.compile(
    r"""
    (?:(?:thumb|thumbnail|left|right|\d+px|upright(?:=[0-9\.]+)?)\|)+
    |^\s*\|.+$
    |^REDIRECT\b""",
    flags=re.DOTALL | re.UNICODE | re.VERBOSE | re.MULTILINE)
pattern_new_lines = re.compile('[\n\r ]+', re.UNICODE)
texts = [wikipedia.documents[i].text for i in range(len(wikipedia.documents))]
texts = [pattern_ignored_words.sub('', texts[i]) for i in range(len(texts))]
texts = [pattern_new_lines.sub(' ', texts[i]) for i in range(len(texts))]
texts = [texts[i].replace("\\", "") for i in range(len(texts))]
texts = [texts[i].replace("\xa0", " ") for i in range(len(texts))]

In [151]:
# Simple wikipedia article texts into single sentences
from nltk import tokenize
sentences = []
sentences += [tokenize.sent_tokenize(texts[i]) for i in range(len(texts))]
#sentences += [texts[i].split(". ") for i in range(len(texts))] #len(texts)
# Now sentences is a list of lists. The next expression flattens it into one long list.
sentences = [item for sublist in sentences for item in sublist]

In [ ]:
pprint.pprint(sentences[0:10])

In [165]:
print(len(sentences))
for i in reversed(range(len(sentences))):
    if len(sentences[i]) < 20 or len(sentences[i]) > 100 \
        or sentences[i][0:9] == "Category:" \
        or sentences[i][0:13] == "Related pages" \
        or sentences[i][0:10] == "References" \
        or sentences[i][0:14] == "Other websites":
        sentences.pop(i)
print(len(sentences))

#Gallery

681193
648280


In [ ]:
pprint.pprint(sentences[530000:530300])

In [ ]:
import statistics
sentence_lengths = [len(sentences[i]) for i in range(len(sentences))]
print(statistics.median(sentence_lengths))

from collections import defaultdict
appearances = defaultdict(int)

sentence_lengths.sort()

for curr in sentence_lengths:
    appearances[curr] += 1
    
a = set(sentence_lengths) 
for i in a:
    print("{} - {}".format(i, appearances[i]))

In [5]:
alphabets = {'a' : 0, 'b': 1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 'o':14,
            'p':15, 'q':16, 'r':17, 's':18, 't':19, 'u':20, 'v':21, 'w':22, 'x':23, 'y':24, 'z':25, 
            '0':26, '1':27, '2':28, '3':29, '4':30, '5':31, '6':32, '7':33, '8':34, '9':35, 
            ' ':36, ',':37, '.':38, ':':39, ';':40, '"':41, "'":42, '':43, '(':44, ')':45} #43 = unknown symbol

idxs = [alphabets[ch] if ch in alphabets else 43 for ch in 'az 123#']

idxs

one_hot = tf.one_hot(idxs, depth=len(alphabets), dtype=tf.uint8)

sess = tf.InteractiveSession()
one_hot.eval()

[0, 25, 36, 27, 28, 29, 43]

In [179]:
sentences_idxs = []
for i in range(len(sentences)):
    idx = []
    for j in sentences[i]:
        if j in alphabets:
            idx += [alphabets[j]]
        else:
            idx += [43]
    sentences_idxs.append(idx)
    
sentences_onehot = [tf.one_hot(sentences_idxs[i], depth=len(alphabets), dtype=tf.uint8) for i in range(len(sentences_idxs))]

In [190]:
sentences_onehot = [tf.one_hot(sentences_idxs[i], depth=len(alphabets), dtype=tf.uint8) for i in range(10000)]

In [192]:
sentences_onehot[0]

<tf.Tensor 'one_hot_420541:0' shape=(61, 44) dtype=uint8>

In [193]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Bidirectional, Reshape,Flatten, Lambda, Input, BatchNormalization
from keras.optimizers import SGD, adam
from keras.layers import TimeDistributed, Dropout
from keras.activations import relu

# Define CTC loss
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    # hack for load_model
    import tensorflow as tf

    ''' from TF: Input requirements
    1. sequence_length(b) <= time for all b
    2. max(labels.indices(labels.indices[:, 1] == b, 2)) <= sequence_length(b) for all b.
    '''

    # print("CTC lambda inputs / shape")
    # print("y_pred:",y_pred.shape)  # (?, 778, 30)
    # print("labels:",labels.shape)  # (?, 80)
    # print("input_length:",input_length.shape)  # (?, 1)
    # print("label_length:",label_length.shape)  # (?, 1)


    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def ds1_dropout(input_dim=26, fc_size=2048, rnn_size=512, dropout=[0.1, 0.1, 0.1], output_dim=29):
    """ DeepSpeech 1 Implementation with Dropout
    Architecture:
        Input MFCC TIMEx26
        3 Fully Connected using Clipped Relu activation function
        3 Dropout layers between each FC
        1 BiDirectional LSTM
        1 Dropout applied to BLSTM
        1 Dropout applied to FC dense
        1 Fully connected Softmax
    Details:
        - Uses MFCC's rather paper's 80 linear spaced log filterbanks
        - Uses LSTM's rather than SimpleRNN
        - No translation of raw audio by 5ms
        - No stride the RNN
    Reference:
        https://arxiv.org/abs/1412.5567
    """
    from keras.utils.generic_utils import get_custom_objects
    get_custom_objects().update({"clipped_relu": clipped_relu})
    K.set_learning_phase(1)

    # Creates a tensor there are usually 26 MFCC
    input_data = Input(name='the_input', shape=(None, input_dim))  # >>(?, max_batch_seq, 26)

    # First 3 FC layers
    init = random_normal(stddev=0.046875)
    x = TimeDistributed(Dense(fc_size, name='fc1', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(input_data)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)
    x = TimeDistributed(Dense(fc_size, name='fc2', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(x)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)
    x = TimeDistributed(Dense(fc_size, name='fc3', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(x)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)

    # Layer 4 BiDirectional RNN
    x = Bidirectional(LSTM(rnn_size, return_sequences=True, activation=clipped_relu, dropout=dropout[1],
                                kernel_initializer='he_normal', name='birnn'), merge_mode='sum')(x)


    # Layer 5+6 Time Dist Dense Layer & Softmax
    # x = TimeDistributed(Dense(fc_size, activation=clipped_relu, kernel_initializer=init, bias_initializer=init))(x)
    x = TimeDistributed(Dropout(dropout[2]))(x)
    y_pred = TimeDistributed(Dense(output_dim, name="y_pred", kernel_initializer=init, bias_initializer=init, activation="softmax"), name="out")(x)

    # Change shape
    labels = Input(name='the_labels', shape=[None,], dtype='int32')
    input_length = Input(name='input_length', shape=[1], dtype='int32')
    label_length = Input(name='label_length', shape=[1], dtype='int32')

    # Keras doesn't currently support loss funcs with extra parameters
    # so CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred,
                                                                       labels,
                                                                       input_length,
                                                                       label_length])

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    return model
